# Contenido

1. Desglose general de montos
2. Conteos por región
3. Montos por tipo de instrumento
4. Conteo por región + % de sector

In [ ]:
import pandas as pd 
import altair as alt
#from thefuzz import fuzz
# el archivo está descargado y actualizado al 15/11/2021
df = pd.read_excel("../../data/portafolio_innova.xlsx")

In [ ]:
df.columns

In [ ]:
# Filtro por regiones y foco_apoyo
zona = ["Región del Biobío", "Región del Libertador General Bernardo O'Higgins","Región del Maule", "Región de Ñuble"]
df = df.query("foco_apoyo == 'Desarrolla innovación con I+D' and regionejecucion in @zona")
# para mas adelante
df['area_ocde'] = df['sector_equivalente']
df['area_anid'] = df['sector_equivalente']

In [ ]:
pd_1 = df.groupby(['anio_adjudicacion', 'tipo_intervencion','sostenible'])\
      .agg({'codigo': 'count','aprobado_corfo':'sum', 'monto_consolidado_ley':'sum'})\
      .reset_index().copy()
pd_1['monto'] = pd_1['monto_consolidado_ley']+pd_1['aprobado_corfo']
pd_1 = pd_1.drop(['monto_consolidado_ley','aprobado_corfo'], 1)
# Convertimos a monto en miles de millones
pd_1['monto'] = pd_1['monto']/1000000000 #solo para visualización
# subset con todos los proyectos
pd_1a = pd_1.groupby(['anio_adjudicacion', 'tipo_intervencion'])\
      .agg({'codigo': 'sum','monto':'sum'})\
      .reset_index().copy()
#Gráfico estándar
chart =  alt.Chart().mark_bar().encode(
            x = alt.X('tipo_intervencion:O',title=None,axis=alt.Axis(labels=False)),
            y = alt.Y('monto:Q', title='Miles de millones'),
            color = alt.Color('tipo_intervencion', title='Intervención') # as third parameter use {,scale=alt.Scale(scheme='accent')}
          )
text = chart.mark_text(align='center', baseline='middle', dy=-10).\
                encode(text = alt.Text("monto", format=".2"))

In [ ]:
# misma tabla de montos segmentados en subsidio y ley
# se puede extrapolar a otros tipos de segmentaciones, como área o región por ejemplo
alt.layer(chart, text, data=pd_1a).facet(column=alt.Column('anio_adjudicacion', title=None), spacing=2).properties(
    title=['Portafolio desarrollo innovación con I+D','Subsidio y Ley de I+D','']
).configure_legend(titleFontSize=14,labelFontSize=10).configure_title(
    fontSize=20, font='Arial', anchor='middle',color='black'
  )

In [ ]:
# utilizamos solo proyectos sostenibles
alt.layer(chart, text, data=pd_1[pd_1['sostenible']=='Sí']).facet(column=alt.Column('anio_adjudicacion', title=None), spacing=2).properties(
    title=['Portafolio desarrollo innovación con I+D','Proyectos Sostenibles','Subsidio y Ley de I+D','']
).configure_legend(titleFontSize=14,labelFontSize=10).configure_title(
    fontSize=20, font='Arial', anchor='middle',color='black'
  )

In [ ]:
# Obtener valores por región
pd_2 = df.groupby(['regionejecucion'])\
      .agg({'codigo': 'count'})\
      .copy()
pd_2['tasa'] = pd_2['codigo'] / pd_2['codigo'].sum() *100
pd_2

In [ ]:
# calculo de montos por instrumentos y total
pd_3 = df.groupby(['regionejecucion'])\
        .agg({'monto_consolidado_ley':'sum','aprobado_corfo':'sum','aprobado_privado_pecuniario':'sum'}).copy()
# Convertimos a monto
conversion = 1000000000
pd_3['monto_consolidado_ley'] = pd_3['monto_consolidado_ley']/conversion
pd_3['aprobado_corfo'] = pd_3['aprobado_corfo']/conversion
pd_3['aprobado_privado_pecuniario'] = (pd_3['aprobado_privado_pecuniario']/conversion)

print('Montos x subsidio: $',pd_3['aprobado_corfo'].sum()*conversion)
print('Montos x ley de I+D: $',pd_3['monto_consolidado_ley'].sum()*conversion)
print('Montos x privados: $',int(pd_3['aprobado_privado_pecuniario'].sum())*conversion)

pd_3['tasa_corfo'] = pd_3['aprobado_corfo'] / pd_3['aprobado_corfo'].sum() *100
pd_3['tasa_ley'] = pd_3['monto_consolidado_ley'] / pd_3['monto_consolidado_ley'].sum() *100
pd_3['tasa_privado'] = pd_3['aprobado_privado_pecuniario'] / pd_3['aprobado_privado_pecuniario'].sum() *100

pd_3

In [ ]:
pd_4 = df.groupby(['regionejecucion', 'sector_equivalente'])\
      .agg({'codigo': 'count'})\
      .copy()
pd_4['TASA'] = pd_4.groupby(level=0).apply(lambda x: x/x.sum())
pd_4 = pd_4.reset_index().sort_values('codigo',ascending=False)
pd_4.head()

In [ ]:
chart4 =  alt.Chart(pd_4).mark_bar().encode(
            x = alt.X('codigo',title=None),
            y = alt.Y('sector_equivalente',title=None,sort=None),
            color = alt.Color('regionejecucion',
                      title='Región',scale=alt.Scale(scheme='dark2')),
          )
chart4.properties(
    title=['Portafolio desarrollo innovación con I+D','Sectores de Desarrollo','']
).configure_legend(titleFontSize=14,labelFontSize=10).configure_title(
    fontSize=20, font='Arial', anchor='middle',color='black'
  )

In [ ]:
pd_4.groupby(['sector_equivalente']).agg({'codigo':'sum'}).sort_values('codigo', ascending=False)

In [ ]:
df_aux2 = df.groupby(['sector_equivalente'])\
          .agg({'codigo':'count','monto_consolidado_ley':'sum','aprobado_corfo':'sum'})\
          .sort_values('aprobado_corfo', ascending=False)
#a miles de millones
df_aux2['monto_consolidado_ley'] = df_aux2['monto_consolidado_ley'] /1000000000
df_aux2['aprobado_corfo'] = df_aux2['aprobado_corfo'] /1000000000
df_aux2

In [ ]:
df['sector_equivalente'].unique()

In [ ]:
# no se que pueden significar estos datos
# son los otros 2 tipos de montos asociados a cada postulación
aux = df.groupby(['regionejecucion'])\
        .agg({'aprobado_privado_pecuniario':'sum','aprobado_privado':'sum'}).copy()
aux

In [ ]:
source = df
base = alt.Chart(source).transform_aggregate(
    count='count()',
    groupby=['regionejecucion', 'sector_equivalente']
).encode(
    alt.X('regionejecucion:O',title=None, scale=alt.Scale(paddingInner=0)),
    alt.Y('sector_equivalente:O',title=None, scale=alt.Scale(paddingInner=0)),
)
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='goldorange'),
        legend=alt.Legend(direction='horizontal')
    )
)
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.condition(
        alt.datum.count < 0.5,
        alt.value('black'),
        alt.value('white')
    )
)
heatmap + text

# PReguntas Alexis

In [ ]:
#5. ¿En qué áreas se está invirtiendo más en la región los últimos 3 años?

df_pg5 = df[df['anio_adjudicacion'].isin([2019,2020,2021])].copy() #se puede modificar años
df_pg5 = df_pg5.groupby(['sector_equivalente','area_ocde','area_anid','anio_adjudicacion'])\
          .agg({'codigo':'count','monto_consolidado_ley':'sum'}).reset_index()
df_pg5 = df_pg5
alt.Chart(df_pg5).mark_bar().encode(
  x = alt.X('area_ocde:O',title=None),
  y = alt.Y('monto_consolidado_ley:Q', title='Miles de millones'),
  color = alt.Color('area_ocde', title='Área OCDE'), # as third parameter use {,scale=alt.Scale(scheme='accent')}
  column = 'anio_adjudicacion'
)

In [ ]:
def SetAreaOCDE(sector):
    if sector == 'Agrícola (excepto vitivinicola)':
        return 'Ciencias Agrícolas'
    elif sector == 'Alimentos (excepto vitivinicola)':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Servicios de ingeniería o de conocimiento':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Manufactura de metales básicos':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Química, caucho y plásticos (excepto industria farmaceutica)':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Minería y metalurgia extractiva':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Construcción':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Farmaceutica':
        return 'Ciencias médicas, Ciencias de la salud'
    elif sector == 'Comercio y retail':
        return 'Ciencias Sociales'
    elif sector == 'Pesca y acuicultura':
        return 'Ciencias Agrícolas'
    elif sector == 'Servicios empresariales administrativos y de apoyo':
        return 'Ciencias Sociales'
    elif sector == 'Forestal':
        return 'Ciencias Agrícolas'
    elif sector == 'Gestión de desechos':   #NO SÉ COMO CLASIFICARLO MEJOR
        return 'Gestión de desechos'
    elif sector == 'Educación':
        return 'Ciencias Sociales'
    elif sector == 'Ganadero':
        return 'Ciencias Agrícolas'
    elif sector == 'Manufactura de maquinaria y equipos (Metalmecánico)':
        return 'Ingeniería, Tecnología ...'
    elif sector ==  'Tecnologías de la información':
        return 'Ingeniería, Tecnología ...'       #PODRIA QUEDAR COMO CIENCIAS NATURALES
                                                  # según: https://vrid.udec.cl/sites/default/files/Areas%20de%20Investigación%20OCDE_0.pdf#overlay-context=formularioseinstructivos
    elif sector == 'Salud y asistencia social':
        return 'Ciencias médicas, Ciencias de la salud'        
    elif sector == 'Logística y Transporte':
        return 'Ingeniería, Tecnología ...' 
    elif sector == 'Recursos Hídricos':
        return 'Ciencias Naturales'
    elif sector == 'Manufactura de no metálicos':
        return 'Ingeniería, Tecnología ...'
    elif sector == 'Energético':
        return 'Ciencias Naturales'
    return sector

def SetAreaANID(sector):
    if sector == 'Agrícola (excepto vitivinicola)':
        return 'Tecnología, Agrícola y Productividad'
    elif sector == 'Alimentos (excepto vitivinicola)':
        return 'Tecnología y ciencias Silvoagropecuarias'
    elif sector == 'Servicios de ingeniería o de conocimiento':
        return 'Tecnología y ciencias de la Ingeniería'
    elif sector == 'Manufactura de metales básicos':
        return 'Tecnología y ciencias de la Ingeniería'
    elif sector == 'Química, caucho y plásticos (excepto industria farmaceutica)':
        return 'Ciencias exactas y naturales'
    elif sector == 'Minería y metalurgia extractiva':
        return 'Ciencias exactas y naturales'
    elif sector == 'Construcción':
        return 'Tecnología y ciencias de la Ingeniería'
    elif sector == 'Farmaceutica':
        return 'Ciencias exactas y naturales'
    elif sector == 'Comercio y retail':
        return 'Ciencias jurídicas, económicas y administrativas'
    elif sector == 'Pesca y acuicultura':
        return 'Tecnología y ciencias Silvoagropecuarias'
    elif sector == 'Servicios empresariales administrativos y de apoyo':
        return 'Ciencias jurídicas, económicas y administrativas'
    elif sector == 'Forestal':
        return 'Tecnología y ciencias Silvoagropecuarias'
    elif sector == 'Gestión de desechos':  
        return 'Ciencias sociales'
    elif sector == 'Educación':
        return 'Ciencias sociales'
    elif sector == 'Ganadero':
        return 'Tecnología y ciencias Silvoagropecuarias'
    elif sector == 'Manufactura de maquinaria y equipos (Metalmecánico)':
        return 'Tecnología y ciencias de la Ingeniería'
    elif sector ==  'Tecnologías de la información':
        return  'Tecnología y ciencias de la Ingeniería'
    elif sector == 'Salud y asistencia social':
        return  'Tecnología y ciencias médicas'
    elif sector == 'Logística y Transporte':
        return 'Tecnología y ciencias de la Ingeniería'
    elif sector == 'Recursos Hídricos':
        return 'Ciencias exactas y naturales'
    elif sector == 'Manufactura de no metálicos':
        return 'Tecnología y ciencias de la Ingeniería'
    elif sector == 'Energético':
        return 'Ciencias jurídicas, económicas y administrativas'
    return sector






df['area_ocde'] = df['area_ocde'].apply(SetAreaOCDE)
df['area_anid'] = df['area_anid'].apply(SetAreaANID)

In [ ]:
df_pg5.to_excel("output.xlsx")  

In [ ]:
df.columns